In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.optimize import minimize

Here I would like to start first with the simple case of a multivariate estimation of two variables observed jointly.


$$ x_1 = N(1,1)  $$


$$ x_2 = 0.25*x_1 + N(0,0.5) $$


In [15]:
#creating a random normal
np.random.seed = 20
x1 = 1 + np.random.normal(0,1,100)
#creating another random normal that is correlated with x1
x2 = 0.25*x1 + np.random.normal(0,0.5,100)

In handleing multivariates it's important to know how to handle matrix multiplication correctly.

The quadratic term appears inside the maximum likelihood estimator is formed through multiplying a row vector times a matrix and then multiplying again by the transpose of the vector. 


The vector (row vector) that contians a joint observation of the multivariate normal looks like this:

$$ X_i = [ x_{1,i} , x_{2,i}] $$


$$ X_i\Sigma X'_i $$


And for N observations it's desirable to compute the following summation:


$$ \sum{X_i\Sigma X'_i} $$

In [12]:
# It can be done through the loop:
X = np.column_stack((x1,x2))
L = np.zeros(100)
for i in range (len(X)):
    h_i =   np.matmul(np.matmul(X[i,],np.cov(x1,x2)),X[i,].reshape(-1,1))
    L[i] = h_i.item()

np.sum(L)

343.54755902032286

Now the next step is to write the Bivariate likelihood:


$$ \ell(\mu,\Sigma) = -\frac{n}{2}\log(2\pi) - \frac{n}{2}\log(|\Sigma|)  - \frac{1}{2} \sum{X_i\Sigma X'_i}  $$


$$\hat \mu = [\hat \mu_{1} , \hat \mu_{2}]$$ 


$$ \hat \Sigma = \begin{bmatrix} 
\hat \sigma_{1}^2  & \hat \sigma_{12} \\
\hat \sigma_{12} & \hat \sigma_{2}^2 
\end{bmatrix}
$$


And the likelihood is maximized with respect to the vector of unknown parameters!

In [6]:
#The likelihood function:
def log_likelihood(p,x1,x2):
    mu1_hat = p[0]
    mu2_hat = p[1]
    sigma1_hat = p[2]
    sigma2_hat = p[3]
    sigma12_hat = p[4]
    mu = np.column_stack((mu1_hat,mu2_hat))
    X = np.column_stack((x1,x2))
    V = np.array([[sigma1_hat,sigma12_hat], [sigma12_hat,sigma2_hat]])
    Det_V = np.linalg.det(V)
    V_inv = np.linalg.inv(V)
    L = np.zeros(len(X))
    for i in range (len(X)):
      h_i =   np.matmul(np.matmul(X[i,] - mu,V_inv),(X[i,]-mu).reshape(-1,1))
      L[i] = h_i.item()

    l = -0.5*(len(X))*np.log(2*np.pi) - 0.5*(len(X))*np.log(Det_V) - 0.5*np.sum(L)
    return -l

In [7]:
p0 = [0.9,0.4,0.7,0.5,0]
ll = log_likelihood(p0,x1,x2)
ll #This is the likelihood of some arbitrary values

130.51613285457668

Before running the optimization on the above function, one would like to mahe sure the diagonal terms on the matrix $ \hat \Sigma $ remains positive throughout the entire optimization iterations.


We want : $$ \hat \sigma_{1}^2 > 0 $$ and $$\hat \sigma_{2}^2 > 0 $$




In [9]:
def constraint(p):
    sigma1_hat = p[2]
    sigma2_hat = p[3]
    return sigma1_hat ,sigma2_hat

cons = { 'type' :'ineq' , 'fun' : constraint}

In [10]:
minimize(log_likelihood, p0, args= (x1,x2),constraints= cons, options={"maxiter": 1000} )

C:\Users\minoa\AppData\Local\Temp\ipykernel_19828\2844859708.py:18: RuntimeWarning: invalid value encountered in log
  l = -0.5*(len(X))*np.log(2*np.pi) - 0.5*(len(X))*np.log(Det_V) - 0.5*np.sum(L)


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 116.88785395465577
       x: [ 1.005e+00  3.007e-01  7.984e-01  3.231e-01  1.868e-01]
     nit: 14
     jac: [ 7.677e-04 -2.972e-03  2.509e-03 -7.340e-03  7.915e-04]
    nfev: 96
    njev: 14

Instead of explicitly passing the restrictions to the solver one can use the Cholesky decomposition on the $\Sigma$ matrix


$$ P = \begin{bmatrix} 
\lambda_{1}  &  0 \\
\lambda_{12} &  \lambda_{2} 
\end{bmatrix}
$$
$$ P' = \begin{bmatrix} 
\lambda_{1}  &  \lambda_{12} \\
0 &  \lambda_{2} 
\end{bmatrix}
$$
$$\Sigma = \begin{bmatrix} 
\lambda_{1}^2  &  \lambda_{1} \lambda_{12} \\
\lambda_{1} \lambda_{12} &  \lambda_{12}^2+\lambda_{2}^2 
\end{bmatrix}
$$


In [25]:
#The likelihood function:
def log_likelihood(p,x1,x2):
    mu1_hat = p[0]
    mu2_hat = p[1]
    lam1_hat = p[2]
    lam2_hat = p[3]
    lam12_hat = p[4]
    mu = np.column_stack((mu1_hat,mu2_hat))
    X = np.column_stack((x1,x2))
    P  = np.array([[lam1_hat,0], [lam12_hat,lam2_hat]])
    P_T = np.array([[lam1_hat,lam12_hat], [0,lam2_hat]])
    V = np.matmul(P,P_T)
    Det_V = np.linalg.det(V)
    V_inv = np.linalg.inv(V)
    L = np.zeros(len(X))
    for i in range (len(X)):
      h_i =   np.matmul(np.matmul(X[i,] - mu,V_inv),(X[i,]-mu).reshape(-1,1))
      L[i] = h_i.item()

    l = -0.5*(len(X))*np.log(2*np.pi) - 0.5*(len(X))*np.log(Det_V) - 0.5*np.sum(L)
    return -l

In [31]:
#Check the likelihood function with any arbitrary guess of the parameters:
p0 = [0,0.9, 1,4,2]
ll = log_likelihood(p0,x1,x2)
ll


370.42127300154607

In [33]:
minimize(log_likelihood, p0, args= (x1,x2), options={"maxiter": 1000} )

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 115.9333899716299
        x: [ 1.132e+00  2.714e-01  8.904e-01 -5.254e-01  7.401e-02]
      nit: 21
      jac: [ 9.537e-07  9.537e-07  0.000e+00 -9.537e-07  1.907e-06]
 hess_inv: [[ 8.123e-03  5.223e-04 ...  2.473e-04  8.362e-06]
            [ 5.223e-04  2.800e-03 ...  3.210e-06 -3.856e-06]
            ...
            [ 2.473e-04  3.210e-06 ...  1.464e-03 -6.347e-06]
            [ 8.362e-06 -3.856e-06 ... -6.347e-06  2.735e-03]]
     nfev: 210
     njev: 35

We can see that the estimates for the mean are immediately reported on the first two elements of the output of the solver.


$$\hat \mu = [ 1.13 , 0.27]$$ 

While for the Variance-Covariance Matrix:

First retrive the $\lambda's$

$$ P = \begin{bmatrix} 
0.89  &  0 \\
0.074 &  -0.525 
\end{bmatrix}




In [34]:
P = np.array([[0.809,0], [0.074,-0.525]])
V = np.matmul(P,P.T)
V

array([[0.654481, 0.059866],
       [0.059866, 0.281101]])

$$\hat \Sigma = PP'=  \begin{bmatrix} 
0.654  &  0.059 \\
0.059 &  0.281 
\end{bmatrix}
$$